In [2]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager

# Setup Chrome options
chrome_options = Options()
chrome_options.add_argument("--headless")  # Run in headless mode
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--window-size=1920,1080")
chrome_options.add_argument("--log-level=3")  # Suppress logs
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("aad your user agent")

# Initialize WebDriver
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

def scrape_amazon(category):
    url = f"https://www.flipkart.com/search?q={category}"
    driver.get(url)
    time.sleep(5)  # Wait for the page to load completely
    
    # Scroll to load more products
    for _ in range(3):  # Adjust the range to load more results
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)
        time.sleep(2)
    
    products = []
    
 
    items = driver.find_elements(By.CSS_SELECTOR, "span._3sdu8W emupdz")
    
    for item in items:
        try:
          
            title = item.find_element(By.CSS_SELECTOR, "span.wjcEIp").text
            
           
            price = item.find_element(By.CSS_SELECTOR, "span.Nx9bqj").text
            
       
            try:
                discount = item.find_element(By.CSS_SELECTOR, "span._3Ay6Sb").text
            except:
                discount = "No Discount"
            
            # If we found both title and price, add to products list
            if title and price:
                products.append({
                    "Category": category,
                    "Title": title,
                    "Price": price,
                    "Discount": discount,
                    "Availability": "In Stock"
                })
        except:
            # Skip items that don't have the required elements
            continue
    
    print(f"Found {len(products)} products for {category}")
    return products

# Categories to scrape
categories = ["electronics", "home appliances", "clothing", "mobiles"]
all_products = []

for category in categories:
    print(f"Scraping {category}...")
    category_products = scrape_flipkart(category)
    all_products.extend(category_products)

# Save data to CSV
df = pd.DataFrame(all_products)
df.to_csv("amazon.csv", index=False)
print(f"Scraping complete. Total {len(all_products)} products. Data saved to flipkart_sales.csv.")

# Close the browser
driver.quit()

In [ ]:
#pip install boto3